## Hadamard Quantum Walks <a id="Hadamard Quantum Walks"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit,QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit_ibm_runtime import Options, Session, SamplerV2 as Sampler
from qiskit.result import marginal_distribution

from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator

In [ ]:
#parameters
n=6 #no. of layers
num_qubit = int(2*(n+1)) #no. of qubits
m= int(n+1) #starting position

#Registers
qr= QuantumRegister(num_qubit, name="qr")
cr= ClassicalRegister(num_qubit,name="cr")
qc= QuantumCircuit(qr,cr)

#Initialize walker at starting position
qc.x(m)

#Initialize starting state of coin |0> 0r |1> or (|0> + i*|1>)/sqrt(2) state
qc.h(0)
qc.s(0)

#Quantum Walk steps
for i in range(n):
        qc.h(0)
        p=int(m-i)
        for j in range(i+1):
            r= int(p + 2*j)
            qc.x(0)
            qc.cswap(0,r-1,r)   # left shift if coin is |0>
            qc.x(0)
            qc.cswap(0,r,r+1)   # Right shift if coin is |1>
            
qc.barrier()

#measurement
for i in range(1,num_qubit):
    qc.measure(qr[i],cr[i])


qc.draw("mpl")

In [ ]:
backend = AerSimulator()

# make quantum circuit compatible to the backend
pm = generate_preset_pass_manager(backend = backend, optimization_level=3)
qc_isa = pm.run(qc)

# run and get counts
sampler = Sampler(mode=backend)
counts = sampler.run([qc_isa], shots = 1000).result()[0].data.cr.get_counts()

plot_distribution(counts)